In [2]:
from __future__ import print_function, division
import sys,os
# line 4 and line 5 below are for development purposes and can be removed
qspin_path = os.path.join(os.getcwd(),"../../")
sys.path.insert(0,qspin_path)

from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.basis import tensor_basis,spinful_fermion_basis_1d,boson_basis_1d # bases
from quspin.tools.measurements import obs_vs_time # calculating dynamics
from quspin.tools.Floquet import Floquet_t_vec # period-spaced time vector
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library
#
##### setting up parameters for simulation
# physical parameters
l = 1
L_f = 3 # system size f
L_b = L_f-1 # system size b
Nf, Nb = 2, l # number of fermions, bosons
N = Nf + Nb # total number of particles
# define time-dependent perturbation

###### create the basis
# build the two bases to tensor together to a bose-fermi mixture
basis_b1=boson_basis_1d(L_b,Nb=[i for i in range(2*l*L_b+1)],sps=2*l+1) # boson basis
basis_b2=boson_basis_1d(L_b,Nb=[i for i in range(2*l*L_b+1)],sps=2*l+1) # boson basis
basis_f=spinful_fermion_basis_1d(L_f,Nf = [(0,0),(0,1),(1,0),(1,1),(2,0),(0,2),(1,2),(2,1),(2,2),(0,3),(3,0),(3,1),(1,3),(2,3),(3,2),(3,3)],double_occupancy=True)
basis=tensor_basis(basis_b1,basis_b2,basis_f) # BFM

############################################## target  hamiltonian
a = 0.1
e = 1.0
m = 0.3
lam = 0.5

bosonic_part_LGT = [[a*e**2/4.,i,i,i,i] for i in range(L_b)] # b^dagger*b^dagger*b*b
mass_f = [[m+1.0/a,i,i] for i in range(L_f)] # f mass term
m_mass_f = [[-m-1.0/a,i,i] for i in range(L_f)] # f mass term

interaction_part_LGT = [[1.0/(a*np.sqrt(l*(l+1))),i,i,i,i+1] for i in range(L_b)] #boson-fermion interaction
m_interaction_part_LGT = [[-1.0/(a*np.sqrt(l*(l+1))),i,i,i,i+1] for i in range(L_b)] #boson-fermion interaction

gauge_violation = [[lam,i,(i+1)] for i in range(L_f-1)] #gauge violating term
m_gauge_violation = [[-lam,i,(i+1)] for i in range(L_f-1)] #gauge violating term


b_0 = [[-0.5*e,0]]
d_0 = [[+0.5*e,0]]
b_1 = [[+0.5*e,1]]
d_1 = [[-0.5*e,1]]
rho_1 = [[-e,1]]
eee = [[e,1]]

static_gauss_law = [
    
    ["n|||", b_0],
    ["|n||", d_0],
    ["n|||", b_1],
    ["|n||", d_1],
    ["||n|", rho_1],
    ["|||n", rho_1],
    ["|I||", eee]

]    


    
    

#
static_target = [
            ["++--|||", bosonic_part_LGT], # b^dagger*b^dagger*b*b
            ["|++--||", bosonic_part_LGT], # b^dagger*b^dagger*b*b

            ["||+|-", mass_f], # f mass term
            ["||-|+", m_mass_f], # f mass term


            ["+|-|+|-", interaction_part_LGT], #boson-fermion interaction
            ["-|+|-|+", m_interaction_part_LGT], #boson-fermion interaction


]
static_violation = [
["||+|-", gauge_violation],
["||-|+", m_gauge_violation]

]

################################################
###### set up Hamiltonian and initial states
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)

H_violation = hamiltonian(static_violation,dynamic_list = [],basis=basis,**no_checks)
H_target = hamiltonian(static_target,dynamic_list = [],basis=basis,**no_checks)
Gauss_law = hamiltonian(static_gauss_law, dynamic_list = [], basis=basis, **no_checks)



In [3]:
# define initial Fock state through strings
s_f = ["111","110","101","011","100","010","001","000"]
s_b = ["20","11","10","02","01","00"]
# basis.index accepts strings and returns the index which corresponds to that state in the basis list
i = np.zeros(8)
i[0] = basis.index(s_b[2],s_b[2],s_f[0],s_f[7]) # find index of product state in basis
i[1] = basis.index(s_b[2],s_b[2],s_f[1],s_f[6])
i[2] = basis.index(s_b[2],s_b[2],s_f[2],s_f[5])
i[3] = basis.index(s_b[2],s_b[2],s_f[3],s_f[4])
i[4] = basis.index(s_b[2],s_b[2],s_f[4],s_f[3])
i[5] = basis.index(s_b[2],s_b[2],s_f[5],s_f[2])
i[6] = basis.index(s_b[2],s_b[2],s_f[6],s_f[1])
i[7] = basis.index(s_b[2],s_b[2],s_f[7],s_f[0])


psi_0 = np.zeros(basis.Ns) # allocate space for state
psi_0[int(i[0])] = 1/(np.sqrt(2)**3) # set MB state to be the given product state
psi_0[int(i[1])] = -1/(np.sqrt(2)**3)
psi_0[int(i[2])] = -1/(np.sqrt(2)**3)
psi_0[int(i[3])] = -1/(np.sqrt(2)**3)
psi_0[int(i[4])] = 1/(np.sqrt(2)**3)
psi_0[int(i[5])] = 1/(np.sqrt(2)**3)
psi_0[int(i[6])] = 1/(np.sqrt(2)**3)
psi_0[int(i[7])] = -1/(np.sqrt(2)**3)


#


In [4]:
def extract_energy(theta,psi,a):
    times0=np.linspace(0.0,theta[0],100)
    times1=np.linspace(0.0,theta[1],100)
    times2=np.linspace(0.0,theta[2],100)
    
    psi_t0 = H_violation.evolve(psi,0.0,times0)
    psi_t1 = H_target.evolve([row[-1] for row in psi_t0],0.0,times1)
    psi_t2 = H_violation.evolve([row[-1] for row in psi_t1],0.0,times2)


    cost_function = np.real(H_target.expt_value([row[-1] for row in psi_t2])) + 100.*np.real(Gauss_law.expt_value([row[-1] for row in psi_t2]))**2
    return cost_function

def extract_state(theta,psi):
    times0=np.linspace(0.0,theta[0],100)
    times1=np.linspace(0.0,theta[1],100)
    times2=np.linspace(0.0,theta[2],100)
    
    psi_t0 = H_violation.evolve(psi,0.0,times0)
    psi_t1 = H_target.evolve([row[-1] for row in psi_t0],0.0,times1)
    psi_t2 = H_violation.evolve([row[-1] for row in psi_t1],0.0,times2)
    
    psi_final = np.array([row[-1] for row in psi_t2])
    return psi_final


NN = 9 #number of iterations
res_theta_0 = np.zeros(NN+1)
res_theta_1 = np.zeros(NN+1)
res_theta_2 = np.zeros(NN+1)

res_theta_0[0] = 0.
res_theta_1[0] = 0.
res_theta_2[0] = 0.

res_energy = np.zeros(NN+1)
res_energy[0] = np.real(H_target.expt_value(psi_0))

psi_ini = np.array([[complex(0.) for i in range(basis.Ns)]for j in range(NN+1)])
psi_ini[0] = psi_0

res_gauss_law = np.array([complex(0.) for i in range(NN+1)])
res_gauss_law[0] = Gauss_law.expt_value(psi_0)

print(extract_state(np.array([res_theta_0[0],res_theta_1[0],res_theta_2[0]]),psi_ini[0]))
#print(res_gauss_law)
print(np.dot(extract_state(np.array([res_theta_0[0],res_theta_1[0],res_theta_2[0]]),psi_ini[0]),psi_0))

[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
(0.9999999999999996+0j)


In [5]:
from scipy.optimize import brute
from scipy.optimize import minimize
brute_ranges = (slice(-0.5,11.,1.),slice(-0.5,11.,1.),slice(-0.5,11.,1.))
for i in range(NN):
    resbrute = 0
    resbrute = brute(extract_energy, brute_ranges, args = (psi_ini[i],1) ,full_output = True ,finish = minimize)
    res_theta_0[i+1] = resbrute[0][0]
    res_theta_1[i+1] = resbrute[0][1]
    res_theta_2[i+1] = resbrute[0][2]
    
    res_energy[i+1] = resbrute[1]
    
    psi_ini[i+1] = extract_state(np.array([resbrute[0][0],resbrute[0][1],resbrute[0][2]]),psi_ini[i])
    res_gauss_law[i+1] = Gauss_law.expt_value(extract_state(np.array([resbrute[0][0],resbrute[0][1],resbrute[0][2]]),psi_ini[i]))

In [6]:
print(res_energy)
print(res_gauss_law)
#print(res_theta_0)
#print(res_theta_1)
#print(res_theta_2)

[ 8.88178420e-16 -2.52704166e+01 -2.55281132e+01 -2.67195191e+01
 -2.67819966e+01 -2.67422951e+01 -2.67704968e+01 -2.69861060e+01
 -2.77204508e+01 -2.77242185e+01]
[0.00000000e+00+0.j 5.55111512e-16+0.j 6.93889390e-16+0.j
 2.77555756e-17+0.j 1.52655666e-16+0.j 5.27355937e-16+0.j
 7.35522754e-16+0.j 4.16333634e-16+0.j 4.44089210e-16+0.j
 4.30211422e-16+0.j]


In [28]:
psi_test = H_target.evolve(psi_ini[9],0.0,np.linspace(0.,0.1,100))
print(np.abs(np.dot([row[-1] for row in psi_test],psi_ini[9])))

0.7714702544378131


In [32]:
#eigenvalues,eigenvectors = H_target.eigh()
#psi_ground = eigenvectors[:,0]
###### time evolve initial state and measure energy
import datetime

def extract_energy(theta,psi,a):
    times0=np.linspace(0.0,theta[0],100)
    times1=np.linspace(0.0,theta[1],100)
    times2=np.linspace(0.0,theta[2],100)
    
    psi_t0 = H_violation.evolve(psi,0.0,times0)
    psi_t1 = H_target.evolve([row[-1] for row in psi_t0],0.0,times1)
    psi_t2 = H_violation.evolve([row[-1] for row in psi_t1],0.0,times2)


    cost_function = np.real(H_target.expt_value([row[-1] for row in psi_t2])) + 100.*np.real(Gauss_law.expt_value([row[-1] for row in psi_t2]))**2
    return cost_function



In [33]:
from scipy.optimize import brute
from scipy.optimize import minimize
print(datetime.datetime.now())
brute_ranges = (slice(-0.5,11.,1.),slice(-0.5,11.,1.),slice(-0.5,11.,1.))
resbrute = brute(extract_energy, brute_ranges, args = (psi_ini[9],1) ,full_output = True ,finish = minimize)
print(datetime.datetime.now())
               

2021-10-27 20:48:05.416016
2021-10-27 20:55:38.574668


In [34]:
print(resbrute[0]) #theta1, theta2

[6.29471492 2.51237643 6.29610475]


In [35]:
print(resbrute[1]) #minimized energy

-27.534189201333692


In [88]:
times0=np.linspace(0.0,resbrute[0][0],100)
times1=np.linspace(0.0,resbrute[0][1],100)
times2=np.linspace(0.0,resbrute[0][2],100)

psi_t0 = H_violation.evolve(psi_0,0.0,times0)
psi_t1 = H_target.evolve([row[-1] for row in psi_t0],0.0,times1)
psi_t2 = H_violation.evolve([row[-1] for row in psi_t1],0.0,times2)

psi_ini_1 = [row[-1] for row in psi_t2]

print(H_target.expt_value(psi_ini_1))

(-25.27041656041346-2.1163626406917047e-16j)


In [89]:
def extract_energy_1(theta,psi,a):
    times0=np.linspace(0.0,theta[0],100)
    times1=np.linspace(0.0,theta[1],100)
    times2=np.linspace(0.0,theta[2],100)
    
    psi_t0 = H_violation.evolve(psi_ini_1,0.0,times0)
    psi_t1 = H_target.evolve([row[-1] for row in psi_t0],0.0,times1)
    psi_t2 = H_violation.evolve([row[-1] for row in psi_t1],0.0,times2)


    cost_function = np.real(H_target.expt_value([row[-1] for row in psi_t2])) + 100.*np.real(Gauss_law.expt_value([row[-1] for row in psi_t2]))**2
    return cost_function

In [90]:
from scipy.optimize import brute
from scipy.optimize import minimize
print(datetime.datetime.now())
brute_ranges = (slice(-0.5,11.,1.),slice(-0.5,11.,1.),slice(-0.5,11.,1.))
resbrute = brute(extract_energy, brute_ranges, args = (psi_ini_1,1) ,full_output = True ,finish = minimize)
print(datetime.datetime.now())
               

2021-10-27 14:32:59.362877
2021-10-27 14:45:15.135597


In [91]:
print(resbrute[0]) #theta1, theta2

[-0.18249443  9.48343837 -0.28308836]


In [92]:
print(resbrute[1]) #minimized energy

-25.528113198669775


In [94]:
times0=np.linspace(0.0,resbrute[0][0],100)
times1=np.linspace(0.0,resbrute[0][1],100)
times2=np.linspace(0.0,resbrute[0][2],100)

psi_t0 = H_violation.evolve(psi_ini_1,0.0,times0)
psi_t1 = H_target.evolve([row[-1] for row in psi_t0],0.0,times1)
psi_t2 = H_violation.evolve([row[-1] for row in psi_t1],0.0,times2)

psi_ini_2 = [row[-1] for row in psi_t2]

print(H_target.expt_value(psi_ini_2))

(-25.528113198669775+1.1102230246251565e-16j)


In [95]:
def extract_energy_2(theta,psi,a):
    times0=np.linspace(0.0,theta[0],100)
    times1=np.linspace(0.0,theta[1],100)
    times2=np.linspace(0.0,theta[2],100)
    
    psi_t0 = H_violation.evolve(psi_ini_2,0.0,times0)
    psi_t1 = H_target.evolve([row[-1] for row in psi_t0],0.0,times1)
    psi_t2 = H_violation.evolve([row[-1] for row in psi_t1],0.0,times2)


    cost_function = np.real(H_target.expt_value([row[-1] for row in psi_t2])) + 100.*np.real(Gauss_law.expt_value([row[-1] for row in psi_t2]))**2
    return cost_function

In [96]:
from scipy.optimize import brute
from scipy.optimize import minimize
print(datetime.datetime.now())
brute_ranges = (slice(-0.5,11.,1.),slice(-0.5,11.,1.),slice(-0.5,11.,1.))
resbrute = brute(extract_energy, brute_ranges, args = (psi_ini_2,1) ,full_output = True ,finish = minimize)
print(datetime.datetime.now())
               

2021-10-27 14:49:14.277021
2021-10-27 14:58:13.948927


In [97]:
print(resbrute[0]) #theta1, theta2

[6.29557685 9.48128187 6.35751863]


In [98]:
print(resbrute[1]) #minimized energy

-26.71951911112678


In [99]:
times0=np.linspace(0.0,resbrute[0][0],100)
times1=np.linspace(0.0,resbrute[0][1],100)
times2=np.linspace(0.0,resbrute[0][2],100)

psi_t0 = H_violation.evolve(psi_ini_2,0.0,times0)
psi_t1 = H_target.evolve([row[-1] for row in psi_t0],0.0,times1)
psi_t2 = H_violation.evolve([row[-1] for row in psi_t1],0.0,times2)

psi_ini_3 = [row[-1] for row in psi_t2]

print(H_target.expt_value(psi_ini_3))

(-26.71951911112678+1.7763568394002505e-15j)


In [100]:
def extract_energy_3(theta,psi,a):
    times0=np.linspace(0.0,theta[0],100)
    times1=np.linspace(0.0,theta[1],100)
    times2=np.linspace(0.0,theta[2],100)
    
    psi_t0 = H_violation.evolve(psi_ini_3,0.0,times0)
    psi_t1 = H_target.evolve([row[-1] for row in psi_t0],0.0,times1)
    psi_t2 = H_violation.evolve([row[-1] for row in psi_t1],0.0,times2)


    cost_function = np.real(H_target.expt_value([row[-1] for row in psi_t2])) + 100.*np.real(Gauss_law.expt_value([row[-1] for row in psi_t2]))**2
    return cost_function

In [101]:
from scipy.optimize import brute
from scipy.optimize import minimize
print(datetime.datetime.now())
brute_ranges = (slice(-0.5,11.,1.),slice(-0.5,11.,1.),slice(-0.5,11.,1.))
resbrute = brute(extract_energy, brute_ranges, args = (psi_ini_3,1) ,full_output = True ,finish = minimize)
print(datetime.datetime.now())
               

2021-10-27 15:02:21.141683
2021-10-27 15:10:40.986207


In [102]:
print(resbrute[0]) #theta1, theta2

[6.29689095 2.51669598 6.28671287]


In [103]:
print(resbrute[1]) #minimized energy

-26.781996618215718


In [111]:
times = np.linspace(0.0,10,100)
times1=np.linspace(0.0,resbrute[0][0],100)
times2=np.linspace(0.0,resbrute[0][1],100)

psi_t1 = H_violation.evolve(psi_ini_3,0.0,times1)
psi_t2 = H_target.evolve([row[-1] for row in psi_t1],0.0,times)
psi_t3 = H_violation.evolve([row[-1] for row in psi_t2],0.0,times2)

psi_ini_4 = [row[-1] for row in psi_t3]

print(H_target.expt_value(psi_ini_4))
print(Gauss_law.expt_value(psi_ini_4))

(-5.695801056250805-8.326672684688674e-17j)
(-1.582067810090848e-15+0j)


In [112]:
def extract_energy_4(theta):
    times1=np.linspace(0.0,theta[0],100)
    times2=np.linspace(0.0,theta[1],100)
    #times3=np.linspace(0.0,theta[2],100)
    #times4=np.linspace(0.0,theta[3],100)
    #times5=np.linspace(0.0,theta[4],100)

    psi_t1 = H_violation.evolve(psi_ini_4,0.0,times1)
    psi_t2 = H_target.evolve([row[-1] for row in psi_t1],0.0,times)
    psi_t3 = H_violation.evolve([row[-1] for row in psi_t2],0.0,times2)
    #psi_t4 = H_target.evolve([row[-1] for row in psi_t3],0.0,times)
    #psi_t5 = H_violation.evolve([row[-1] for row in psi_t4],0.0,times3)
    #psi_t6 = H_target.evolve([row[-1] for row in psi_t5],0.0,times)
    #psi_t7 = H_violation.evolve([row[-1] for row in psi_t6],0.0,times4)
    #psi_t8 = H_target.evolve([row[-1] for row in psi_t7],0.0,times)
    #psi_t9 = H_violation.evolve([row[-1] for row in psi_t8],0.0,times5)


    cost_function = H_target.expt_value([row[-1] for row in psi_t3]) + 100*Gauss_law.expt_value([row[-1] for row in psi_t3])**2
    return np.real(cost_function)

In [113]:
from scipy.optimize import brute
from scipy.optimize import minimize
print(datetime.datetime.now())
brute_ranges = (slice(0,10,1.),slice(0,10,1.))
resbrute = brute(extract_energy_4, brute_ranges,full_output = True ,finish = minimize)
print(datetime.datetime.now())
               

2021-10-26 14:04:32.345071
2021-10-26 14:05:21.263494


In [114]:
print(resbrute[0]) #theta1, theta2

[-0.10002423 -1.18810346]


In [115]:
print(resbrute[1]) #minimized energy

-6.026920088071035


In [116]:
times = np.linspace(0.0,10,100)
times1=np.linspace(0.0,resbrute[0][0],100)
times2=np.linspace(0.0,resbrute[0][1],100)

psi_t1 = H_violation.evolve(psi_ini_4,0.0,times1)
psi_t2 = H_target.evolve([row[-1] for row in psi_t1],0.0,times)
psi_t3 = H_violation.evolve([row[-1] for row in psi_t2],0.0,times2)

psi_ini_5 = [row[-1] for row in psi_t3]

print(H_target.expt_value(psi_ini_5))
print(Gauss_law.expt_value(psi_ini_5))

(-6.026920088071035-2.7755575615628914e-16j)
(-3.552713678800501e-15+0j)


In [117]:
def extract_energy_5(theta):
    times1=np.linspace(0.0,theta[0],100)
    times2=np.linspace(0.0,theta[1],100)
    #times3=np.linspace(0.0,theta[2],100)
    #times4=np.linspace(0.0,theta[3],100)
    #times5=np.linspace(0.0,theta[4],100)

    psi_t1 = H_violation.evolve(psi_ini_5,0.0,times1)
    psi_t2 = H_target.evolve([row[-1] for row in psi_t1],0.0,times)
    psi_t3 = H_violation.evolve([row[-1] for row in psi_t2],0.0,times2)
    #psi_t4 = H_target.evolve([row[-1] for row in psi_t3],0.0,times)
    #psi_t5 = H_violation.evolve([row[-1] for row in psi_t4],0.0,times3)
    #psi_t6 = H_target.evolve([row[-1] for row in psi_t5],0.0,times)
    #psi_t7 = H_violation.evolve([row[-1] for row in psi_t6],0.0,times4)
    #psi_t8 = H_target.evolve([row[-1] for row in psi_t7],0.0,times)
    #psi_t9 = H_violation.evolve([row[-1] for row in psi_t8],0.0,times5)


    cost_function = H_target.expt_value([row[-1] for row in psi_t3]) + 100*Gauss_law.expt_value([row[-1] for row in psi_t3])**2
    return np.real(cost_function)

In [118]:
from scipy.optimize import brute
from scipy.optimize import minimize
print(datetime.datetime.now())
brute_ranges = (slice(0,10,1.),slice(0,10,1.))
resbrute = brute(extract_energy_5, brute_ranges,full_output = True ,finish = minimize)
print(datetime.datetime.now())
               

2021-10-26 14:05:21.656452
2021-10-26 14:06:06.259785


In [119]:
print(resbrute[0]) #theta1, theta2

[ 0.14525717 -1.21860723]


In [120]:
print(resbrute[1]) #minimized energy

-7.605261587719853


In [121]:
times = np.linspace(0.0,10,100)
times1=np.linspace(0.0,resbrute[0][0],100)
times2=np.linspace(0.0,resbrute[0][1],100)

psi_t1 = H_violation.evolve(psi_ini_5,0.0,times1)
psi_t2 = H_target.evolve([row[-1] for row in psi_t1],0.0,times)
psi_t3 = H_violation.evolve([row[-1] for row in psi_t2],0.0,times2)

psi_ini_6 = [row[-1] for row in psi_t3]

print(H_target.expt_value(psi_ini_6))
print(Gauss_law.expt_value(psi_ini_6))

(-7.605261587719853-2.7755575615628914e-17j)
(-1.7208456881689926e-15+0j)


In [122]:
def extract_energy_6(theta):
    times1=np.linspace(0.0,theta[0],100)
    times2=np.linspace(0.0,theta[1],100)
    #times3=np.linspace(0.0,theta[2],100)
    #times4=np.linspace(0.0,theta[3],100)
    #times5=np.linspace(0.0,theta[4],100)

    psi_t1 = H_violation.evolve(psi_ini_6,0.0,times1)
    psi_t2 = H_target.evolve([row[-1] for row in psi_t1],0.0,times)
    psi_t3 = H_violation.evolve([row[-1] for row in psi_t2],0.0,times2)
    #psi_t4 = H_target.evolve([row[-1] for row in psi_t3],0.0,times)
    #psi_t5 = H_violation.evolve([row[-1] for row in psi_t4],0.0,times3)
    #psi_t6 = H_target.evolve([row[-1] for row in psi_t5],0.0,times)
    #psi_t7 = H_violation.evolve([row[-1] for row in psi_t6],0.0,times4)
    #psi_t8 = H_target.evolve([row[-1] for row in psi_t7],0.0,times)
    #psi_t9 = H_violation.evolve([row[-1] for row in psi_t8],0.0,times5)


    cost_function = H_target.expt_value([row[-1] for row in psi_t3]) + 100*Gauss_law.expt_value([row[-1] for row in psi_t3])**2
    return np.real(cost_function)

In [123]:
from scipy.optimize import brute
from scipy.optimize import minimize
print(datetime.datetime.now())
brute_ranges = (slice(0,10,1.),slice(0,10,1.))
resbrute = brute(extract_energy_6, brute_ranges,full_output = True ,finish = minimize)
print(datetime.datetime.now())
               

2021-10-26 14:07:41.650860
2021-10-26 14:08:24.393818


In [124]:
print(resbrute[0]) #theta1, theta2

[-0.00539434  0.07064312]


In [125]:
print(resbrute[1]) #minimized energy

-7.614896406210371


In [127]:
times = np.linspace(0.0,10,100)
times1=np.linspace(0.0,resbrute[0][0],100)
times2=np.linspace(0.0,resbrute[0][1],100)

psi_t1 = H_violation.evolve(psi_ini_6,0.0,times1)
psi_t2 = H_target.evolve([row[-1] for row in psi_t1],0.0,times)
psi_t3 = H_violation.evolve([row[-1] for row in psi_t2],0.0,times2)

psi_ini_7 = [row[-1] for row in psi_t3]

print(H_target.expt_value(psi_ini_7))
print(Gauss_law.expt_value(psi_ini_7))

(-7.614896406210371+0j)
(-1.6653345369377348e-15+0j)


In [128]:
def extract_energy_7(theta):
    times1=np.linspace(0.0,theta[0],100)
    times2=np.linspace(0.0,theta[1],100)
    #times3=np.linspace(0.0,theta[2],100)
    #times4=np.linspace(0.0,theta[3],100)
    #times5=np.linspace(0.0,theta[4],100)

    psi_t1 = H_violation.evolve(psi_ini_7,0.0,times1)
    psi_t2 = H_target.evolve([row[-1] for row in psi_t1],0.0,times)
    psi_t3 = H_violation.evolve([row[-1] for row in psi_t2],0.0,times2)
    #psi_t4 = H_target.evolve([row[-1] for row in psi_t3],0.0,times)
    #psi_t5 = H_violation.evolve([row[-1] for row in psi_t4],0.0,times3)
    #psi_t6 = H_target.evolve([row[-1] for row in psi_t5],0.0,times)
    #psi_t7 = H_violation.evolve([row[-1] for row in psi_t6],0.0,times4)
    #psi_t8 = H_target.evolve([row[-1] for row in psi_t7],0.0,times)
    #psi_t9 = H_violation.evolve([row[-1] for row in psi_t8],0.0,times5)


    cost_function = H_target.expt_value([row[-1] for row in psi_t3]) + 100*Gauss_law.expt_value([row[-1] for row in psi_t3])**2
    return np.real(cost_function)

In [129]:
from scipy.optimize import brute
from scipy.optimize import minimize
print(datetime.datetime.now())
brute_ranges = (slice(0,10,1.),slice(0,10,1.))
resbrute = brute(extract_energy_7, brute_ranges,full_output = True ,finish = minimize)
print(datetime.datetime.now())
               

2021-10-26 14:23:33.096754
2021-10-26 14:24:20.054394


In [130]:
print(resbrute[0]) #theta1, theta2

[0.79753307 5.15726189]


In [131]:
print(resbrute[1]) #minimized energy

-9.259893078770984


In [132]:
times = np.linspace(0.0,10,100)
times1=np.linspace(0.0,resbrute[0][0],100)
times2=np.linspace(0.0,resbrute[0][1],100)

psi_t1 = H_violation.evolve(psi_ini_7,0.0,times1)
psi_t2 = H_target.evolve([row[-1] for row in psi_t1],0.0,times)
psi_t3 = H_violation.evolve([row[-1] for row in psi_t2],0.0,times2)

psi_ini_8 = [row[-1] for row in psi_t3]

print(H_target.expt_value(psi_ini_8))
print(Gauss_law.expt_value(psi_ini_8))

(-9.259893078770984+1.6653345369377348e-16j)
(-4.440892098500626e-16+0j)


In [133]:
def extract_energy_8(theta):
    times1=np.linspace(0.0,theta[0],100)
    times2=np.linspace(0.0,theta[1],100)
    #times3=np.linspace(0.0,theta[2],100)
    #times4=np.linspace(0.0,theta[3],100)
    #times5=np.linspace(0.0,theta[4],100)

    psi_t1 = H_violation.evolve(psi_ini_8,0.0,times1)
    psi_t2 = H_target.evolve([row[-1] for row in psi_t1],0.0,times)
    psi_t3 = H_violation.evolve([row[-1] for row in psi_t2],0.0,times2)
    #psi_t4 = H_target.evolve([row[-1] for row in psi_t3],0.0,times)
    #psi_t5 = H_violation.evolve([row[-1] for row in psi_t4],0.0,times3)
    #psi_t6 = H_target.evolve([row[-1] for row in psi_t5],0.0,times)
    #psi_t7 = H_violation.evolve([row[-1] for row in psi_t6],0.0,times4)
    #psi_t8 = H_target.evolve([row[-1] for row in psi_t7],0.0,times)
    #psi_t9 = H_violation.evolve([row[-1] for row in psi_t8],0.0,times5)


    cost_function = H_target.expt_value([row[-1] for row in psi_t3]) + 100*Gauss_law.expt_value([row[-1] for row in psi_t3])**2
    return np.real(cost_function)

In [134]:
from scipy.optimize import brute
from scipy.optimize import minimize
print(datetime.datetime.now())
brute_ranges = (slice(0,10,1.),slice(0,10,1.))
resbrute = brute(extract_energy_8, brute_ranges,full_output = True ,finish = minimize)
print(datetime.datetime.now())
               

2021-10-26 14:25:39.211427
2021-10-26 14:26:21.434821


In [135]:
print(resbrute[0]) #theta1, theta2

[-0.01296797 -0.36437937]


In [136]:
print(resbrute[1]) #mnimized energy

-9.605648945865187


In [137]:
times = np.linspace(0.0,10,100)
times1=np.linspace(0.0,resbrute[0][0],100)
times2=np.linspace(0.0,resbrute[0][1],100)

psi_t1 = H_violation.evolve(psi_ini_8,0.0,times1)
psi_t2 = H_target.evolve([row[-1] for row in psi_t1],0.0,times)
psi_t3 = H_violation.evolve([row[-1] for row in psi_t2],0.0,times2)

psi_ini_9 = [row[-1] for row in psi_t3]

print(H_target.expt_value(psi_ini_9))
print(Gauss_law.expt_value(psi_ini_9))

(-9.605648945865187+0j)
(-1.6653345369377348e-15+0j)


In [138]:
def extract_energy_9(theta):
    times1=np.linspace(0.0,theta[0],100)
    times2=np.linspace(0.0,theta[1],100)
    #times3=np.linspace(0.0,theta[2],100)
    #times4=np.linspace(0.0,theta[3],100)
    #times5=np.linspace(0.0,theta[4],100)

    psi_t1 = H_violation.evolve(psi_ini_9,0.0,times1)
    psi_t2 = H_target.evolve([row[-1] for row in psi_t1],0.0,times)
    psi_t3 = H_violation.evolve([row[-1] for row in psi_t2],0.0,times2)
    #psi_t4 = H_target.evolve([row[-1] for row in psi_t3],0.0,times)
    #psi_t5 = H_violation.evolve([row[-1] for row in psi_t4],0.0,times3)
    #psi_t6 = H_target.evolve([row[-1] for row in psi_t5],0.0,times)
    #psi_t7 = H_violation.evolve([row[-1] for row in psi_t6],0.0,times4)
    #psi_t8 = H_target.evolve([row[-1] for row in psi_t7],0.0,times)
    #psi_t9 = H_violation.evolve([row[-1] for row in psi_t8],0.0,times5)


    cost_function = H_target.expt_value([row[-1] for row in psi_t3]) + 100*Gauss_law.expt_value([row[-1] for row in psi_t3])**2
    return np.real(cost_function)

In [139]:
from scipy.optimize import brute
from scipy.optimize import minimize
print(datetime.datetime.now())
brute_ranges = (slice(0,10,1.),slice(0,10,1.))
resbrute = brute(extract_energy_9, brute_ranges,full_output = True ,finish = minimize)
print(datetime.datetime.now())
               

2021-10-26 14:27:22.737206
2021-10-26 14:28:05.560595


In [140]:
print(resbrute[0]) #theta1, theta2

[0.12833032 0.31664947]


In [141]:
print(resbrute[1]) #minimized energy

-9.812094205978507


In [55]:
times = np.linspace(0.0,10,100)
times1=np.linspace(0.0,resbrute[0][0],100)
times2=np.linspace(0.0,resbrute[0][1],100)

psi_t1 = H_violation.evolve(psi_ini,0.0,times1)
psi_t2 = H_target.evolve([row[-1] for row in psi_t1],0.0,times)
psi_t3 = H_violation.evolve([row[-1] for row in psi_t2],0.0,times2)

#psi_ini = [row[-1] for row in psi_t3]

Gauss_law.expt_value([row[-1] for row in psi_t3])

(1.6653345369377348e-15+0j)

In [9]:
Gauss_law.__init__(static_gauss_law, dynamic_list = [],basis=basis, check_herm = True)
#for checking the Hermiticity of the operators

Hermiticity check passed!


<ipython-input-9-741e28dd489c>:1: UserWarning: Test for symmetries not implemented for <class 'quspin.basis.tensor.tensor_basis'>, to turn off this warning set check_symm=False in hamiltonian
  Gauss_law.__init__(static_gauss_law, dynamic_list = [],basis=basis, check_herm = True)
<ipython-input-9-741e28dd489c>:1: UserWarning: Test for particle conservation not implemented for <class 'quspin.basis.tensor.tensor_basis'>, to turn off this warning set check_pcon=False in hamiltonian
  Gauss_law.__init__(static_gauss_law, dynamic_list = [],basis=basis, check_herm = True)


In [10]:
Gauss_law.expt_value(psi_0)

0j

In [ ]:
psi_ini = 

In [35]:
#eigenvalues,eigenvectors = H_target.eigh()
#psi_ground = eigenvectors[:,0]
###### time evolve initial state and measure energy
import datetime

times = np.linspace(0.0,10,100)
def extract_energy(theta):
    times1=np.linspace(0.0,theta[0],100)
    times2=np.linspace(0.0,theta[1],100)
    #times3=np.linspace(0.0,theta[2],100)
    #times4=np.linspace(0.0,theta[3],100)
    #times5=np.linspace(0.0,theta[4],100)

    psi_t1 = H_violation.evolve(psi_0,0.0,times1)
    psi_t2 = H_target.evolve([row[-1] for row in psi_t1],0.0,times)
    psi_t3 = H_violation.evolve([row[-1] for row in psi_t2],0.0,times2)
    #psi_t4 = H_target.evolve([row[-1] for row in psi_t3],0.0,times)
    #psi_t5 = H_violation.evolve([row[-1] for row in psi_t4],0.0,times3)
    #psi_t6 = H_target.evolve([row[-1] for row in psi_t5],0.0,times)
    #psi_t7 = H_violation.evolve([row[-1] for row in psi_t6],0.0,times4)
    #psi_t8 = H_target.evolve([row[-1] for row in psi_t7],0.0,times)
    #psi_t9 = H_violation.evolve([row[-1] for row in psi_t8],0.0,times5)


    cost_function = H_target.expt_value([row[-1] for row in psi_t3]) + 100*Gauss_law.expt_value([row[-1] for row in psi_t3])**2
    return np.real(cost_function)

print(datetime.datetime.now())
extract_energy(np.array([10.,10.]))
print(datetime.datetime.now())


2021-10-25 12:27:32.364856
2021-10-25 12:27:32.719826


In [ ]:
from scipy.optimize import brute
from scipy.optimize import minimize
print(datetime.datetime.now())
brute_ranges = (slice(0,10,1.),slice(0,10,1.))
resbrute = brute(extract_energy, brute_ranges,full_output = True ,finish = minimize)
print(datetime.datetime.now())
              

In [15]:
from scipy.optimize import minimize
import datetime
print(datetime.datetime.now())
initial_guess = np.array([0.,0.])
nelder_mead = minimize(extract_energy, initial_guess,method='Nelder-Mead')
print(datetime.datetime.now())


2021-10-25 12:02:52.938388
2021-10-25 12:03:55.670169


In [16]:
nelder_mead.x

array([-3.12792599,  4.38588058])

In [20]:
nelder_mead.fun

-4.506222272167034

In [161]:
print(basis.Ns)

5184


(1+0j)

In [46]:
theta = np.array([1.,1.,1.])
psi = psi_0

In [50]:
times0=np.linspace(0.0,theta[0],100)
times1=np.linspace(0.0,theta[1],100)
times2=np.linspace(0.0,theta[2],100)

psi_t0 = H_violation.evolve(psi,0.0,times0)
psi_t1 = H_target.evolve([row[-1] for row in psi_t0],0.0,times1)
psi_t2 = H_violation.evolve([row[-1] for row in psi_t1],0.0,times2)
print(np.dot(psi,[row[-1] for row in psi_t2]))

(-0.09332855054364825+0.5504184972904382j)
